<a href="https://colab.research.google.com/github/STRMNGnwo/CT5133-DeepLearning/blob/main/DeepLearningAssignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CT5133 - Assignment 1

  -Submitted by Srinivas Ilancheran (19280039) and Lukasz Szemet (19502109) of the MSc Artificial Intelligence programme



# Assignment Sections:

### Section 1

**Instructions:**

Implement Logistic Regression (Topic 2):
  1. Use Jupyter Notebook (Python or R) to implement a neural network approach to logistic regression (no hidden layers, one output node)
  2. Your code should follow my notes to implement the algorithm from scratch.
  3. Your notebook should include a brief description of the algorithm, with all references.
  4. Your code must handle different numbers of inputs and different numbers of training cases, but you don't have to support more than one binary output node

Part 1 notes

Class node
4 functions
contructor(input then init weights), activation(sigmoid function), weighted sum(w .  x + b), forward


In [ ]:
import numpy as np
import sys
import pandas as pd

In [64]:
class LogisticRegressionNeuron:

  def __init__(self,data_shape):


    self.weights=np.random.rand(*data_shape)

    #self.weights=np.squeeze(self.weights)
    '''the squeeze is done to change the shape from 1,3 to just 3, (Which is the shape of the input)
    this is necessary as in the dot product function, we're doing input * weight (input is of shape 3,0 which would make dot product with 1,3 shape impossible)

    An easier fix would simply be changing the order of dot product to be weight* input (1,3 can be multiplied with shape 3,0)

    '''
    print("Initial weights matrix:", self.weights)
    print("Shape of Initial weights matrix:", self.weights.shape)
    # initialising a bias value
    self.bias=np.random.rand(1,1)


  def weighted_sum_func(self,input):

    return np.dot(self.weights,input) + self.bias


  def activation_func(self,input):

      return 1/(1+np.exp(-input))


  def forward(self,input):

    #defining a single pass

    #send the input into the weighted sum function, to get W . input

    weighted_sum=self.weighted_sum_func(input)

    #send the weighted_sum into the activation function (sigmoid) to get either a 0, or a 1 (based on threshold value)
    activation_output=self.activation_func(weighted_sum)

    return activation_output



In [107]:
def train_model(model,training_data_X,training_data_Y, training_params):

  # SGD parameters
  epochs=training_params["epochs"]
  max_iterations= len(training_data_X)*epochs

  print("Max iterations: ",max_iterations)
  learning_rate=training_params["lr"]

  #probability threshold:
  threshold=0.5

  samples=training_data_X.to_numpy()
  labels=training_data_Y.to_numpy()

  i=0
  prev_loss=0
  running_loss=0
  #Training loop to perform SGD:
  while i<max_iterations:
    #randomly choose a training sample from samples
    rand_sample_index=np.random.randint(len(samples))

    input_sample=np.array(samples[rand_sample_index])

    #print("Shape of input sample:",input_sample.shape)

    #input_sample -> for the should be of shape (1,num.attributes) as its a single row containing num.attributes columns

    #model output -> probability
    probability=model.forward(input_sample)

    #convert probability into label
    predicted_label=1 if probability>0.5 else 0

    #calculate loss using Stochastic Gradient Descent
    # input_sample[1] is true class value (Actually, a sample's attributes-> samples[rand_sample], true class value-> labels[rand_sample])
    curr_loss = -((labels[rand_sample_index]*np.log(probability)) + (1-labels[rand_sample_index]) * np.log(1-probability))

    delta_w = np.zeros(model.weights.shape)

    for idx, weight in enumerate(model.weights):
      delta_w[idx] = (probability - labels[rand_sample_index])*input_sample[idx]

    delta_b = (probability - labels[rand_sample_index])

    for idx, weight in enumerate(model.weights):
      model.weights[idx] -= learning_rate * delta_w[idx]

    model.bias -= learning_rate * delta_b

    i += 1
    running_loss += curr_loss

    if i%len(samples)==0:
      print(f"{i/len(samples)} Epochs finished")
      if (running_loss-prev_loss) < 10**(-12): #latter condition is to check for convergence (change in weights is minimal)
        print(f"Converged! after {i} iterations ")
        i = max_iterations
      prev_loss = running_loss
      running_loss = 0


### Section 2

**Instructions:**

• I will supply two fairly small datasets:
  – One will be linearly separable (almost or fully), the other will not
  – I will provide sample Python code to load and plot the datasets; you are
allowed to use this code in your own assignment

• Divide each dataset randomly into:
  – Training set (70%):use for main training
  – Validation set (15%): use for tuning, e.g. selecting learning rates
  – Test set (15%): held out set for final performance evaluation

• Train a logistic regressor using your code from
Part 1, and see how it performs on both datasets

• In your notebook, summarise results and
provide observations and conclusions


In [108]:
#Using pandas to read in the csv
blobs_dataset=pd.read_csv("/content/blobs400.csv")
circles_dataset=pd.read_csv("/content/circles500.csv")

print("Blobs dataset:")
print(blobs_dataset.head())
print("\n\nCircles dataset:")
print(circles_dataset.head())

Blobs dataset:
         X1        X2        X3  Class
0  1.418221  2.124375 -0.433905      1
1  1.590404  0.935434  1.510369      1
2  2.311458 -1.026668  1.031930      1
3  1.186782  0.591894  0.563649      1
4  1.661888  4.047231  0.987049      0


Circles dataset:
         X0        X1  Class
0  0.180647  0.552945      1
1 -0.188674  0.325629      1
2  0.413742  0.931251      0
3 -0.199223  0.902665      0
4  0.488279 -0.341202      1


In [111]:
training_blobs = blobs_dataset.sample(frac=0.7, random_state=69)
training_circles = circles_dataset.sample(frac=0.7, random_state=69)

training_blobs = blobs_dataset.sample(frac=0.7, random_state=69)
training_circles = circles_dataset.sample(frac=0.7, random_state=69)

#dividing the attributes and the label in the training blobs training dataset
training_blobs_X=training_blobs.loc[:, ["X1","X2","X3"]]
training_blobs_Y= training_blobs.loc[:,["Class"]]

print(training_blobs.iloc[[0]].shape)

print(training_blobs_X.shape)

print(training_blobs_Y.shape)

(1, 4)
(280, 3)
(280, 1)


In [117]:
#initialising the LogisiticRegression Neuron
LR_Model=LogisticRegressionNeuron(np.array(training_blobs_X.iloc[[0]]).shape)

#setting training params
train_params={ "epochs":10000,"lr":0.000001}

#training the model on the blobs dataset's training partition
train_model(LR_Model,training_blobs_X,training_blobs_Y, train_params)

Initial weights matrix: [[0.06984089 0.35381306 0.14937838]]
Shape of Initial weights matrix: (1, 3)
Max iterations:  2800000
1.0 Epochs finished
2.0 Epochs finished
3.0 Epochs finished
4.0 Epochs finished
Converged! after 1120 iterations 


In [118]:
print("New weights matrix value:", LR_Model.weights)

New weights matrix value: [[0.06886319 0.35283536 0.14840068]]


### Evaluating the model's accuracy on the datasets

### Section 3

### Section 4

### Section 5